In [1]:
import matplotlib
matplotlib.use('TkAgg')  # Use TkAgg backend for matplotlib
from matplotlib import pyplot as plt
from PIL import Image
import torch
from transformers import GLPNImageProcessor, GLPNForDepthEstimation



In [2]:
feature_extractor = GLPNImageProcessor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")



preprocessor_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/245M [00:00<?, ?B/s]

In [3]:
image = Image.open("../data/food.jpg")
new_height = 480 if image.height > 480 else image.height
new_height -=(new_height % 32)  # Ensure height is a multiple of 32
new_width =int(new_height * image.width / image.height)
diff = new_width % 32

new_width = new_width - diff  if diff < 16 else new_width + 32 - diff
new_size = (new_width, new_height)
image = image.resize(new_size)

In [ ]:
inputs = feature_extractor(images=image, return_tensors="pt")


In [5]:
with torch.no_grad():
    outputs = model(**inputs)
    
    predicted_depth = outputs.predicted_depth

In [6]:
pads =15
output = predicted_depth.squeeze().cpu().numpy() * 1000.0
output = output[pads:-pads, pads:-pads]
image = image.crop((pads, pads, image.width - pads, image.height - pads))


In [8]:
fig, ax = plt.subplots(1, 2)
ax[0].imshow(image)
ax[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
ax[1].imshow(output, cmap='plasma')
ax[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.tight_layout()
plt.pause(5)

In [9]:
import numpy as np
import open3d as o3d



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [13]:
width, height = image.size

depth_image =(output *255 /np.max(output)).astype(np.uint8)
image = np.array(image)

depth_o3d = o3d.geometry.Image(depth_image)
image_o3d = o3d.geometry.Image(image)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(image_o3d, depth_o3d,convert_rgb_to_intensity=False)


In [15]:
camera_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera_intrinsic.set_intrinsics(width, height, 525.0, 525.0, width / 2, height / 2)

In [18]:
pcd_raw =o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image,camera_intrinsic)

invalid command name "128123663892928delayed_destroy"
    while executing
"128123663892928delayed_destroy"
    ("after" script)
invalid command name "128123399215360delayed_destroy"
    while executing
"128123399215360delayed_destroy"
    ("after" script)
invalid command name "128123710515328delayed_destroy"
    while executing
"128123710515328delayed_destroy"
    ("after" script)


In [17]:
o3d.visualization.draw_geometries([pcd_raw])

In [19]:
cl,ind = pcd_raw.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd = pcd_raw.select_by_index(ind)



In [20]:
pcd.estimate_normals()
pcd.orient_normals_to_align_with_direction()

In [21]:
o3d.visualization.draw_geometries([pcd])

In [22]:
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10,n_threads=1)[0]

rotation = mesh.get_rotation_matrix_from_xyz((np.pi, 0, 0))
mesh.rotate(rotation, center=(0, 0, 0))


TriangleMesh with 570213 points and 1140501 triangles.

In [ ]:
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

TypeError: draw_geometries(): incompatible function arguments. The following argument types are supported:
    1. (geometry_list: list[open3d.cuda.pybind.geometry.Geometry], window_name: str = 'Open3D', width: int = 1920, height: int = 1080, left: int = 50, top: int = 50, point_show_normal: bool = False, mesh_show_wireframe: bool = False, mesh_show_back_face: bool = False, lookat: Optional[numpy.ndarray[numpy.float64[3, 1]]] = None, up: Optional[numpy.ndarray[numpy.float64[3, 1]]] = None, front: Optional[numpy.ndarray[numpy.float64[3, 1]]] = None, zoom: Optional[float] = None) -> None

Invoked with: [TriangleMesh with 570213 points and 1140501 triangles.]; kwargs: mesh_show_back_face=True, mesh_show_wireframe=True, mesh_show_vertex_label=False